# Focalplane Reconstruction

This notebook works on exploring algorithms for focalplane reconstruction.

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy import units as u
from scipy import signal
from scipy.ndimage import gaussian_filter

import toast
import toast.ops
from toast.observation import default_values as defaults
from toast.tests._helpers import create_ground_data


## Fake Dataset

Create a fake focalplane and observation.

In [ ]:
data = create_ground_data(
    None,
    sample_rate=100.0 * u.Hz,
    el_nod=False,
    pixel_per_process=7,
    fknee=1.0 * u.Hz,
)
obzero = data.obs[0]

In [ ]:
# Pointing operators
sim_detpoint_azel = toast.ops.PointingDetectorSimple(
    boresight=defaults.boresight_azel, quats="quats_azel"
)
sim_detpoint_radec = toast.ops.PointingDetectorSimple(
    boresight=defaults.boresight_radec, quats="quats_radec"
)
sim_weights_azel = toast.ops.StokesWeights(
    mode="I",
    detector_pointing=sim_detpoint_azel,
)
sim_weights_radec = toast.ops.StokesWeights(
    mode="I",
    detector_pointing=sim_detpoint_radec,
)
sim_pixels = toast.ops.PixelsWCS(
    projection="TAN",
    detector_pointing=sim_detpoint_radec,
    auto_bounds=True,
    resolution=(0.1 * u.degree, 0.1 * u.degree),
    pixels="sim_pixels",
    submaps=1,
)

sim_detpoint_azel.apply(data)
sim_detpoint_radec.apply(data)
sim_weights_azel.apply(data)
sim_weights_radec.apply(data)
sim_pixels.apply(data)

In [ ]:
# Pixel Distribution and Hits
sim_dist = toast.ops.BuildPixelDistribution(
    pixel_dist="dist_sim",
    pixel_pointing=sim_pixels,
)
sim_dist.apply(data)

sim_hits = toast.ops.BuildHitMap(
    hits="sim_hits",
    pixel_dist=sim_dist.pixel_dist,
    pixels=sim_pixels.pixels,
)
sim_hits.apply(data)

In [ ]:
print(data[sim_dist.pixel_dist])
print(data[sim_hits.hits])
print(sim_pixels.wcs)

print(data[sim_hits.hits].data[0, :, 0])

fig = plt.figure(figsize=[12, 8])
ax = fig.add_subplot(1, 1, 1)
ax.imshow(data[sim_hits.hits].data[0, :, 0].reshape(sim_pixels.wcs_shape))
plt.show()


In [ ]:
# Make a fake sky with a spike

center_indx = (
    sim_pixels.wcs_shape[0] // 2,
    sim_pixels.wcs_shape[1] // 2,
)

fake_sky = np.zeros(sim_pixels.wcs_shape, dtype=np.float64)
fake_sky[center_indx] = 10000.0

fake_sky = gaussian_filter(fake_sky, 3.0)

fig = plt.figure(figsize=[12, 8])
ax = fig.add_subplot(1, 1, 1)
ax.imshow(fake_sky)
plt.show()

data["fake_sky"] = toast.pixels.PixelData(
    data[sim_dist.pixel_dist],
    np.float64,
    n_value=1,
    units=u.K,
)
data["fake_sky"].data[0, :, 0] = fake_sky.flatten()

In [ ]:
toast.ops.Reset(detdata=["signal"]).apply(data)

In [ ]:
# Default noise model
toast.ops.DefaultNoiseModel().apply(data)

In [ ]:
# Scan from fake sky
toast.ops.ScanMap(
    pixels=sim_pixels.pixels,
    weights=sim_weights_radec.weights,
    map_key="fake_sky",
).apply(data)


In [ ]:
# Add atmosphere sim
sim_atm = toast.ops.SimAtmosphere(
    detector_pointing=sim_detpoint_azel,
    detector_weights=sim_weights_azel,
).apply(data)

In [ ]:
# Timeconstant convolution
#toast.ops.TimeConstant(tau=1.0 * u.second).apply(data)

In [ ]:
# Add simulated noise
toast.ops.SimNoise().apply(data)

In [ ]:
for det in obzero.local_detectors:
    fig = plt.figure(figsize=[12, 8])
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(
        obzero.shared["times"].data,
        obzero.detdata["signal"][det],
        color="black",
        label="Detector Data",
    )
    ax.set_xlabel("Time")
    ax.set_ylabel("K")
    ax.legend(loc="best")
    plt.show()

## Filters



In [ ]:
toast.ops.CommonModeFilter(det_flag_mask=0).apply(data)

In [ ]:
filt_sos = signal.butter(
    3, 
    [0.01, 5.0], 
    btype="bandpass",
    output="sos", 
    fs=obzero.telescope.focalplane.sample_rate.to_value(u.Hz)
)

In [ ]:
filt_tod = signal.sosfilt(filt_sos, obzero.detdata["signal"][0])

In [ ]:
slc = slice(-2000, -1, 1)
#slc = slice(None)
for det in obzero.local_detectors:
    fig = plt.figure(figsize=[12, 8])
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(
        obzero.shared["times"].data[slc],
        obzero.detdata["signal"][det, slc],
        color="black",
        label="Detector Data",
    )
    ax.plot(
        obzero.shared["times"].data[slc],
        filt_tod[slc],
        color="red",
        label="Filtered Detector Data",
    )

    ax.set_xlabel("Time")
    ax.set_ylabel("K")
    ax.legend(loc="best")

    #ax.set_xlim(200, 300)
    plt.show()